<a href="https://colab.research.google.com/github/plaban1981/Langchain_usecases/blob/main/Advanced_RAG_Using_Llama_index_llama3_qdrant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install required libraries

In [3]:
%%writefile requirements.txt
llama-index
llama-index-llms-huggingface
llama-index-embeddings-fastembed
llama-parse
fastembed
chromadb
python-dotenv
langchain-groq
chainlit
fastembed
unstructured[md]
llama-index-vector-stores-qdrant
einops
accelerate

Overwriting requirements.txt


In [46]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [47]:
!pip install -U llama-index-vector-stores-qdrant

  Attempting uninstall: llama-index-vector-stores-qdrant
    Found existing installation: llama-index-vector-stores-qdrant 0.2.6
    Uninstalling llama-index-vector-stores-qdrant-0.2.6:
      Successfully uninstalled llama-index-vector-stores-qdrant-0.2.6


In [38]:
!pip install torch sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 4.9 MB/s eta 0:00:00


In [4]:
!pip install -r requirements.txt

  Using cached llama_index-0.10.31-py3-none-any.whl (6.9 kB)
  Using cached llama_index_llms_huggingface-0.1.4-py3-none-any.whl (7.2 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 58.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 79.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 90.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 112.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 1

## Download Data

In [1]:
!mkdir Data

mkdir: cannot create directory ‘Data’: File exists


In [6]:
! wget "https://arxiv.org/pdf/1810.04805.pdf" -O Data/arxiv.pdf

--2024-04-24 15:59:29--  https://arxiv.org/pdf/1810.04805.pdf
Resolving arxiv.org (arxiv.org)... 151.101.131.42, 151.101.3.42, 151.101.67.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.131.42|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 775166 (757K) [application/pdf]
Saving to: ‘Data/arxiv.pdf’

Data/arxiv.pdf      100%[===================>] 757.00K  --.-KB/s    in 0.03s   

2024-04-24 15:59:29 (27.0 MB/s) - ‘Data/arxiv.pdf’ saved [775166/775166]



## Load the Documents

In [2]:
# load documents
from llama_index.core import SimpleDirectoryReader
documents = SimpleDirectoryReader("/content/Data").load_data()

In [3]:
print(len(documents))

16


## Instantiate Embedding Model

In [4]:
from llama_index.embeddings.fastembed import FastEmbedEmbedding
from llama_index.core import Settings
#
embed_model = FastEmbedEmbedding(model_name="BAAI/bge-small-en-v1.5")
#
Settings.embed_model = embed_model
#
Settings.chunk_size = 512
#

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

## Define System Prompt

In [5]:
from llama_index.core import PromptTemplate


system_prompt = "You are a Q&A assistant. Your goal is to answer questions as accurately as possible based on the instructions and context provided."


# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = PromptTemplate("<|USER|>{query_str}<|ASSISTANT|>")

## Instantiate LLM

In [6]:
from huggingface_hub import notebook_login
notebook_login()

In [7]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from llama_index.llms.huggingface import HuggingFaceLLM
tokenizer = AutoTokenizer.from_pretrained(
    "meta-llama/Meta-Llama-3-8B-Instruct"

)

stopping_ids = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>"),
]

llm = HuggingFaceLLM(
    context_window=8192,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.7, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="meta-llama/Meta-Llama-3-8B-Instruct",
    model_name="meta-llama/Meta-Llama-3-8B-Instruct",
    device_map="auto",
    stopping_ids=stopping_ids,
    tokenizer_kwargs={"max_length": 4096},
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16}
)

Settings.llm = llm
Settings.chunk_size = 512

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
documents[0]

Document(id_='30a76ebe-1a6e-409c-a344-df41b413fc7c', embedding=None, metadata={'page_label': '1', 'file_name': 'arxiv.pdf', 'file_path': '/content/Data/arxiv.pdf', 'file_type': 'application/pdf', 'file_size': 775166, 'creation_date': '2024-04-24', 'last_modified_date': '2019-05-28'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='BERT: Pre-training of Deep Bidirectional Transformers for\nLanguage Understanding\nJacob Devlin Ming-Wei Chang Kenton Lee Kristina Toutanova\nGoogle AI Language\n{jacobdevlin,mingweichang,kentonl,kristout }@google.com\nAbstract\nWe introduce a new language representa-\ntion model called BERT , which stands for\nBidirectional Encoder Representations from\nTransformers. Unlike recent language repre-\nsentation models (Peters et a

## Instantiate the Vectorstore - Qdrant

In [9]:
import qdrant_client
from IPython.display import Markdown, display
from llama_index.core import VectorStoreIndex
from llama_index.core import StorageContext
from llama_index.vector_stores.qdrant import QdrantVectorStore

In [10]:
client = qdrant_client.QdrantClient(
    # you can use :memory: mode for fast and light-weight experiments,
    # it does not require to have Qdrant deployed anywhere
    # but requires qdrant-client >= 1.1.1
    location=":memory:"
    # otherwise set Qdrant instance address with:
    # url="http://<host>:<port>"
    # otherwise set Qdrant instance with host and port:
    #host="localhost",
    #port=6333
    # set API KEY for Qdrant Cloud
    #api_key="I3F5XEYYHL0b9lK0XsCojohzvtGbu_BULF2HIjHxRVJOITk7nuJg2w",
)
#
vector_store = QdrantVectorStore(client=client, collection_name="test")
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context,
)

* https://github.com/run-llama/llama_index/issues/13090

* https://github.com/run-llama/llama_index/issues/13091

## ReRanker

In [11]:
from llama_index.core.postprocessor import SentenceTransformerRerank

rerank = SentenceTransformerRerank(
    model="cross-encoder/ms-marco-MiniLM-L-2-v2", top_n=3
)

In [12]:
import time
query_engine = index.as_query_engine(
    similarity_top_k=10, node_postprocessors=[rerank]
)

now = time.time()
response = query_engine.query(
    "What is instruction finetuning?",
)
print(f"Response Generated: {response}")
print(f"Elapsed: {round(time.time() - now, 2)}s")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Response Generated:  Instruction fine-tuning is not explicitly mentioned in the provided context. However, based on the text, it can be inferred that fine-tuning is a process where a pre-trained model like BERT is adapted to a specific task by swapping out the appropriate inputs and outputs. This process is described as "straightforward" and allows BERT to model many downstream tasks by fine-tuning all the parameters end-to-end.
Elapsed: 7.32s


In [13]:

now = time.time()
response = query_engine.query(
    "Describe the Feature-based Approach with BERT??",
)
print(f"Response Generated: {response}")
print(f"Elapsed: {round(time.time() - now, 2)}s")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Response Generated:  According to the text, the Feature-based Approach with BERT involves extracting the activations from one or more layers of BERT without fine-tuning any parameters of BERT. These contextual embeddings are then used as input to a randomly initialized two-layer 768-dimensional BiLSTM before the classification layer. This approach is used to ablate the fine-tuning approach and demonstrate the effectiveness of BERT for both fine-tuning and feature-based approaches.
Elapsed: 6.78s


In [14]:

now = time.time()
response = query_engine.query(
    "What is SQuADv2.0?",
)
print(f"Response Generated: {response}")
print(f"Elapsed: {round(time.time() - now, 2)}s")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Response Generated:  According to the provided context, SQuAD v2.0 is an extension of the SQuAD 1.1 problem definition, allowing for the possibility that no short answer exists in the provided paragraph, making the problem more realistic.
Elapsed: 4.15s
